<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_LinealMultiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión lineal Múltiple

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Nueva sección

In [11]:
data1 = pd.read_csv('/content/drive/My Drive/Dataset/50_Startups.csv',delimiter=';')
x = data1.iloc [:,:-1].values
y = data1.iloc [:,4].values

In [ ]:
pd.DataFrame(x)

### Codificar datos categórico

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder= LabelEncoder()
x[:,3] = labelencoder.fit_transform(x[:,3])
ct = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories='auto'),[3])],remainder='passthrough')
### remainder='passthrough'. deja el resto de las columnas en pié
x=np.array(ct.fit_transform(x),dtype=np.int)

In [ ]:
pd.DataFrame(x)

### Evitar la trampa de las variables Dummy

In [13]:
x = x[:,1:]

### Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

# Escalado de variables
# en este ejemplo no es necesario lo dejaremos comentariado
#from sklearn.preprocessing import StandardScaler
#sc_x = StandardScaler()
#x_train = sc_x.fit.transform(x_train)
#x_test = sc_x.fit.transform(x_test)

### Ajustar el modelo de Regresión Lineal Múltiple con el conjunto de entrenamiento

In [15]:
from sklearn.linear_model import LinearRegression
regresion = LinearRegression()
regresion.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### predicción de resultados con el conjunto de testing

In [16]:
y_pred = regresion.predict(x_test)

In [ ]:
pd.DataFrame(y_pred)

In [ ]:
pd.DataFrame(x)

### Construir el modelo óptimo de RLM utilizando la eliminación hacia átras
### MÍNIMOS CUADRADOS ORDINARIOS OLS
### forma manual

In [ ]:
import statsmodels.api as sm
x = np.append(arr = np.ones((50,1)).astype(int), values=x, axis=1)
sl= 0.05
x_opt = x[:,[0,1,2,3,4,5]]
regresion_OLS = sm.OLS (endog=y,exog=x_opt).fit()
regresion_OLS.summary()

In [ ]:
pd.DataFrame(x_opt)

In [ ]:
x_opt = x[:,[0,2,3,4,5]]
regresion_OLS = sm.OLS (endog = y, exog = x_opt).fit()
regresion_OLS.summary()


In [ ]:
x_opt = x[:,[0,3,4,5]]
regresion_OLS = sm.OLS(endog=y, exog=x_opt).fit()
regresion_OLS.summary()

In [ ]:
x_opt= x[:,[0,3,5]]
regresion_OLS = sm.OLS(endog=y, exog=x_opt).fit()
regresion_OLS.summary()

In [ ]:
x_opt=x[:,[0,3]]
regresion_OLS= sm.OLS(endog=y, exog=x_opt).fit()
regresion_OLS.summary()

In [ ]:
pd.DataFrame(x_opt)

### METODO ELIMINACION HACIA ATRAS. Forma AUTOMATICA

In [37]:
import statsmodels.api as sm
def elimnaAtras (x,SL):
  numvar = len(x[0])
  for i in range(0 , numvar):
    regresor_ols = sm.OLS(y, x.tolist()).fit()
    maxvar=max(regresor_ols.pvalues).astype(float)
    if maxvar>SL:
      for j in range(0,numvar-i):
        if (regresor_ols.pvalues[j].astype(float)== maxvar):
          x=np.delete(x,j,1)
  regresor_ols.summary()
  return x
SL=0.05
x_opt = x[:,[0,1,2,3,4,5]]        
x_modelo = elimnaAtras(x_opt, SL)

In [ ]:
pd.DataFrame(x_modelo)